In [1]:
%load_ext tensorboard
import numpy as np
import tensorflow as tf
import datetime

In [2]:
# clear logs
!rm -rf ./logs/custom_metrics/

In [3]:
# creating regression data for the function y = 0.5x + 2

data_size = 1000
train_pct = 0.8

train_size = int(data_size * train_pct)

x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]


In [4]:
logdir = "logs/custom_metrics/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

file_writer = tf.summary.create_file_writer(logdir + "/learning_rate")
file_writer.set_as_default()

def lr_schedule(epoch):
  
    learning_rate = 0.2
    if epoch > 10:
        learning_rate = 0.02
    if epoch > 20:
        learning_rate = 0.01
    if epoch > 50:
        learning_rate = 0.005

    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [5]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(16))
model.add(tf.keras.layers.Dense(1))

model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(lr=0.2))


In [6]:
training_history = model.fit(
    x_train,
    y_train,
    batch_size=800,
    epochs=100,
    verbose=0,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback, lr_callback],
)


In [7]:
%tensorboard --logdir logs/custom_metrics